<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_04%ED%8C%8C%EC%9D%B4%EC%8D%AC(%ED%81%AC%EB%A1%A4%EB%A7%813%2Cwordcloud%2C_selenium).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
[문제179] 네이버의 언론사별 랭킹뉴스 데이터를 수집해주세요.
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
from pandas import Series, DataFrame
import pandas as pd
import re
import time

url = 'https://news.naver.com/main/ranking/popularDay.naver'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')

# 랭킹뉴스 첫 번째 언론사의 첫 번째 기사 제목과 url test
url = soup.select_one('div.rankingnews_box > ul.rankingnews_list > li > div.list_content > a')['href']

# url 들어가서 기사 제목, text 추출 test
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')

soup.select_one('h2.media_end_head_headline').text # 제목
soup.select_one('div#dic_area')# text(내용)



# 랭킹 언론사별 5개의 뉴스 url 추출 

url =  'https://news.naver.com/main/ranking/popularDay.naver'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')

s = soup.select('ul.rankingnews_list > li > a')
total_url=[]
for i in s:
    total_url.append(i['href'])

total_url

# url마다 기사 제목, text 추출
news_df = DataFrame(columns=['title','contents'])
for i in total_url:
    html = urlopen(i)
    soup = BeautifulSoup(html,'html.parser')
    
    # 제목
    title = soup.select_one('h2.media_end_head_headline').text 
    
    # 내용text
    contents = soup.select_one('div#dic_area')
    
    news_df = news_df.append({'title':title,'contents':contents},ignore_index=True)
    time.sleep(2)
    
####### 강사님
url = 'https://news.naver.com/main/ranking/popularDay.naver'
html = urlopen(url)
soup = beautifulSoup(html,'html.parser')
div_rank = soup.select('div.rankingnews_box')
len(div_rank)

# 첫 번째 언론사
div_rank[0].select_one('strong.rankingnews_name').text # 언론사 이름
div_rank[0].select('ul > li > div > a')[0].string
div_rank[0].select('ul > li > div > a')[0]['href'] #url 추출
div_rank[0].select('ul > li > div > a')[1].string
div_rank[0].select('ul > li > div > a')[1]['href'] #url 추출

# 두 번째 언론사
div_rank[1].select_one('strong.rankingnews_name').text # 언론사 이름
div_rank[1].select('ul > li > div > a')[0].string
div_rank[1].select('ul > li > div > a')[0]['href'] #url 추출
div_rank[1].select('ul > li > div > a')[1].string
div_rank[1].select('ul > li > div > a')[1]['href'] #url 추출

html = urlopen(div_rank[0].select('ul > li > div > a')[0]['href'])
soup = BeautifulSoup(html,'html.parser')

soup.select_one('div#newsct_article > div#dic_area').text

# 결과
url = 'https://news.naver.com/main/ranking/popularDay.naver'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')
div_rank = soup.select('div.rankingnews_box')


news_df = DataFrame(columns=['press_name','title','contents'])
for i in range(0,len(div_rank)+1):
    press_name = div_rank[i].select_one('strong.rankingnews_name').text
    for j in range(0,6):
        html = urlopen(div_rank[i].select('ul > li > div > a')[j]['href'])
        soup = BeautifulSoup(html,'html.parser')
        # 제목
        title = soup.select_one('h2.media_end_head_headline').text 
        # 내용
        contents = soup.select_one('div#newsct_article > div#dic_area').text
        
        news_df = news_df.append({'press_name':press_name,'title':title,'contents':contents},ignore_index=True)
        time.sleep(2)

# 최종 ( 2개 언론사만 추출 다 추출하려면 시간이 오래걸림)
url = 'https://news.naver.com/main/ranking/popularDay.naver?mid=etc&sid1=111'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')

news_ranking_df = DataFrame(columns=['언론사','랭킹URL','title','news'])    

for i in soup.select('div.rankingnews_box')[:2]:
    press = i.select_one('strong.rankingnews_name').text
    news_ranking=[]
    news_ranking_url = []
    news = []
    
    for j in i.select('ul > li > div > a'):
        news_ranking.append(j.string)
        news_ranking_url.append(j['href'])
        html = urlopen(j['href'])
        soup = BeautifulSoup(html,'html.parser')
        news.append(soup.select_one('div#newsct_article > div#dic_area').text.strip())
        time.sleep(2)
    news_ranking_df = news_ranking_df.append(DataFrame({'언론사':press,'랭킹URL':news_ranking_url,'title':news_ranking,'news':news}),ignore_index=True)
    print('수집 언론사 : ',press)
    time.sleep(2)
    
news_ranking_df.to_csv('c:/data/news_ranking.csv',index=False) # 파일로 저장


[문제180] 수집한 뉴스 기사 내용에서 [내용..] 출력해주세요.
re.findall('\[.+\]',news_ranking_df['news'][0])

#강사님
re.findall('\[.+?\]',news_ranking_df.news[0])

news_ranking_df.news.apply(lambda x : re.findall('\[.+?\]', x))

[문제181] 수집한 뉴스 기사 내용에서 <내용..> 출력해주세요.
re.findall('\<.+\>',news_ranking_df['news'][0])

news_ranking_df.news.apply(lambda x : re.findall('\<.+?\>', x))

[문제182] 수집한 뉴스 기사 내용에서 (내용..) 출력해주세요.
re.findall('\([가-힣A-z]+[^\)]*\)',news_ranking_df['news'][1])

news_ranking_df.news.apply(lambda x : re.findall('\(.+?\)', x))

[문제183] 네이버 웹툰 인기순을 딕셔너리 자료형에 수집해주세요.
comic_dict = {}
url = 'https://comic.naver.com/index'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')

li = soup.select('ol#realTimeRankFavorite >li')
comic_dict[li[0]['class']] = li[0].select_one('a').text
comic_dict[li[0]['class'][0]] = li[0].select_one('a').text
comic_dict[''.join(li[0]['class'])] = li[0].select_one('a').text

li[0]['class'] -> str변환?
li[0]['class'][0]
''.join(li[0]['class'])

for i in li:
    comic_dict[''.join(i['class'])] = i.select_one('a').text

dict([(key.replace('rank',''),value)for key,value in comic_dict.items()])



############################################33##########3
★ wordcloud

■ wordcloud 수동으로 받아서 설치
'''
https://www.lfd.uci.edu/~gohlke/pythonlibs/#wordcloud
(base) C:\WINDOWS\system32>python --version
Python 3.9.7

(base) C:\WINDOWS\system32>cd C:\Users\ehdgh\anaconda3\Lib

(base) C:\Users\ehdgh\anaconda3\Lib>dir *.whl
 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: FC92-4C2B

 C:\Users\ehdgh\anaconda3\Lib 디렉터리

2022-04-04  오후 02:17           161,093 wordcloud-1.8.1-cp39-cp39-win_amd64.whl
               1개 파일             161,093 바이트
               0개 디렉터리  106,958,286,848 바이트 남음

(base) C:\Users\ehdgh\anaconda3\Lib>pip install wordcloud-1.8.1-cp39-cp39-win_amd64.whl
Processing c:\users\ehdgh\anaconda3\lib\wordcloud-1.8.1-cp39-cp39-win_amd64.whl
Requirement already satisfied: pillow in c:\users\ehdgh\anaconda3\lib\site-packages (from wordcloud==1.8.1) (8.4.0)
Requirement already satisfied: matplotlib in c:\users\ehdgh\anaconda3\lib\site-packages (from wordcloud==1.8.1) (3.4.3)
Requirement already satisfied: numpy>=1.6.1 in c:\users\ehdgh\anaconda3\lib\site-packages (from wordcloud==1.8.1) (1.20.3)
Requirement already satisfied: kiwisolver>=1.0.1 in c:\users\ehdgh\anaconda3\lib\site-packages (from matplotlib->wordcloud==1.8.1) (1.3.1)
Requirement already satisfied: python-dateutil>=2.7 in c:\users\ehdgh\anaconda3\lib\site-packages (from matplotlib->wordcloud==1.8.1) (2.8.2)
Requirement already satisfied: cycler>=0.10 in c:\users\ehdgh\anaconda3\lib\site-packages (from matplotlib->wordcloud==1.8.1) (0.10.0)
Requirement already satisfied: pyparsing>=2.2.1 in c:\users\ehdgh\anaconda3\lib\site-packages (from matplotlib->wordcloud==1.8.1) (3.0.4)
Requirement already satisfied: six in c:\users\ehdgh\anaconda3\lib\site-packages (from cycler>=0.10->matplotlib->wordcloud==1.8.1) (1.16.0)
Installing collected packages: wordcloud
Successfully installed wordcloud-1.8.1
'''
import matplotlib.pylab as plt
from matplotlib import font_manager,rc
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/HMKMAMI.TTF').get_name()
rc('font',family=font_name)
from wordcloud import WordCloud
word = {'떡볶이':100,'감자탕':50,'순대국밥':10,'치즈':70,'볶음밥':80,'김밥':20,'짜장면':300,'야끼만두':150,'치킨':100,
        '핫도그':90,'호두':60,'알감자':50,'짬뽕':200,'탕수육':150}

w = WordCloud(font_path='C:/Windows/Fonts/HMKMAMI.TTF',
              background_color='white',
              width=900,height=500).generate_from_frequencies(word)
plt.imshow(w) # wordcloud 보여주기
plt.axis('off') # 축 없애기

news_ranking_df.news

txt = ' '.join(news_ranking_df.news)
txt
w = WordCloud(font_path='C:/Windows/Fonts/HMKMAMI.TTF',
              background_color='white',
              width=900,height=500).generate(txt) # generate : 텍스트를 단어들로 변환해줌
plt.imshow(w) # wordcloud 보여주기
plt.axis('off') # 축 없애기


# 영화 감상평 wordcloud
movie = DataFrame(columns=['day','id','review','point'])
for j in range(1,11):
        
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=190695&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='+str(j) 
    # 영화 code만 바꾸면 다른 영화들의 평도 다 가져올 수 있다
    html = urlopen(url)
    soup = BeautifulSoup(html,'html.parser')
    
    review = []
    for i in soup.select('div.score_reple > p'):
        if i.select_one('span.ico_viewer') == None:
            review.append(i.text.strip())
        else:
            i.select_one('span.ico_viewer').extract()
            review.append(i.text.strip())
    
    point = []
    for i in soup.select('div.star_score > em'):
        point.append(i.text)
    
    id = []
    for i in soup.select('div.score_reple > dl > dt > em > a > span'):
        id.append(i.text)
    
    day = []
    for i in soup.select('div.score_reple > dl > dt > em:nth-of-type(2)'):
        day.append(i.text)
    
    
    movie = movie.append(DataFrame({'day':day,'id':id,'review':review,'point':point}))
    
    time.sleep(3)
    
txt = ' '.join(movie.review)
w = WordCloud(font_path='C:/Windows/Fonts/HMKMAMI.TTF',
              background_color='white').generate(txt)
plt.imshow(w)
plt.axis('off')

[문제184] winter.txt 텍스트를 이용해서 워드클라우드 생성해주세요. # 겨울왕국 대사
import numpy as np
from PIL import Image

alice_mask = np.array(Image.open("c:/data/alice_mask.png"))
with open('c:/data/winter.txt','r') as file:
    winter = file.read()

len(winter)

w = WordCloud(font_path='c:/Windows/Fonts/HMKMAMI.TTF',
          background_color='white',
          colormap='twilight',
          mask=alice_mask,
          contour_width=3,
          contour_color='steelblue').generate(winter)
plt.imshow(w)
plt.axis('off')

###################
★ selenium
웹 브라우저를 컨트롤하여 웹을 자동화 하는 도구

# selenium 설치 ( anaconda 관리자 실행 )
(base) C:\Users\ehdgh\anaconda3\Lib>pip install selenium

from selenium import webdriver

url = "https:/www.naver.com"
driver = webdriver.Chrome("c:/data/chromedriver.exe") #여기서 오류가 나면 크롬 드라이버 다운받고 원하는 곳에 저장
driver.get(url) # url에 접속
driver.save_screenshot("c:/data/naver.png") # 스크린샷 저장
driver.quit() # 종료


driver = webdriver.Chrome("c:/data/chromedriver.exe")
url = 'https://comic.naver.com/index'
driver.get(url) 
html = driver.page_source
driver.quit()

comic_dict = {}
soup = BeautifulSoup(html,'html.parser')

li = soup.select('ol#realTimeRankFavorite >li')

for i in li:
    comic_dict[''.join(i['class'])] = i.select_one('a').text

from selenium.webdriver.common.by import By

url = "https:/www.naver.com"
driver = webdriver.Chrome("c:/data/chromedriver.exe") #여기서 오류가 나면 크롬 드라이버 다운받고 원하는 곳에 저장
driver.get(url) # url에 접속
driver.implicitly_wait(2) # time.sleep과 같은 기능
driver.find_element(By.CLASS_NAME,'id_finance').click() # 태그 요소 클릭하기
html = driver.page_source # 현재 접속 페이지 소스 추출하기
soup = BeautifulSoup(html,'html.parser')
soup.select('p')
driver.quit() # 종료

url = "https://search.naver.com/search.naver?where=image"
driver = webdriver.Chrome("c:/data/chromedriver.exe")
driver.get(url) # url에 접속
driver.implicitly_wait(2) # time.sleep과 같은 기능
element = driver.find_element(By.CLASS_NAME,'box_window')
element.clear()
element.send_keys('강아지')
driver.implicitly_wait(2)
element.submit()

import time
from selenium.webdriver.common.keys import Keys

driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
driver.find_element(By.TAG_NAME,'body').send_keys(Keys.HOME)
driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_UP)

for i in range(5):
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(2)

html = driver.page_source
driver.quit()

soup = BeautifulSoup(html,'html.parser')
soup
img_url = []
for i in  soup.select('img._image'):
    img_url.append(i.attrs['src'])
    
len(img_url)
img_url[0]

import urllib.request as req 
# 이미지 파일 url을 이용하여 다운받는 함수 urlretrieve
req.urlretrieve(img_url[0],'c:/img/python_img/1.jpg')


[문제]다음 강아지 이미지 다운
url = 'https://search.daum.net/search?w=img'
driver = webdriver.Chrome('c:/data/chromedriver.exe')
driver.get(url)
driver.implicitly_wait(2)
element = driver.find_element(By.CLASS_NAME,'tf_keyword')
element.clear()
element.send_keys('강아지') # 검색창에 강아지 단어 입력
element.submit() # 수행

for i in range(4):
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(2)

html = driver.page_source
driver.quit()
soup = BeautifulSoup(html,'html.parser')

img_url = []
for i in soup.select('img.thumb_img'):
    img_url.append(i.attrs['src'])

num = 1
for i in img_url:
    req.urlretrieve(i,'c:/img/python_img/dog'+str(num)+'.png')
    num += 1

#
driver.find_element(By.CLASS_NAME,'expender open').click() # 태그 요소 클릭하기
